In [162]:
import pandas as pd

import numpy as np

%matplotlib inline

import pickle

import visdom

from tqdm import tnrange, tqdm_notebook
from tqdm.autonotebook import tqdm
tqdm.pandas()

from time import time,sleep
from datetime import datetime

from sklearn.metrics.pairwise import cosine_similarity
# import visdom
# vis=visdom.Visdom()
# env="TagBased"

import re

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# %matplotlib inline

In [2]:
list_industry=["水泥","食品飲料","石化","紡織","電機機械","電器電纜","化學工業",
               "建材居家用品","造紙","鋼鐵金屬","車輛相關","科技相關","營建地產","運輸","觀光休閒娛樂",
               "金融相關","百貨通路","公用事業","控股","生技醫療保健","農林漁牧","航天軍工","能源","傳播出版","綜合",
               "傳產其他","其他","金屬礦採選",
              ]



### Load dict_industry

In [14]:
with open("D:dict_industry","rb") as f:
    dict_industry=pickle.load(f)

In [15]:
dict_industry["金融相關"]

{'保險經紀',
 '再保險',
 '壽險',
 '房地產投資信託',
 '投信',
 '投資信託',
 '消費金融',
 '產險',
 '票券',
 '租賃',
 '證券',
 '證金公司',
 '資產管理',
 '金控',
 '金融其他',
 '金融業',
 '金融相關',
 '銀行'}

# Load Vip NEWS with industrytag

In [16]:
with open("D:News_vip_with_industrytag","rb") as f:
    News_vip_with_industrytag=pickle.load(f)

In [17]:
News_vip_with_industrytag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504261 entries, 0 to 504260
Data columns (total 13 columns):
publishtime      504261 non-null datetime64[ns]
title            504261 non-null object
body             504261 non-null object
relateid         504261 non-null object
guid             504261 non-null object
no_use           331542 non-null object
tags             331542 non-null object
title_token      504261 non-null object
body_token       504261 non-null object
from             504261 non-null int32
all_1            504261 non-null int32
tags_cn          504261 non-null object
indusrty_tags    504261 non-null object
dtypes: datetime64[ns](1), int32(2), object(10)
memory usage: 46.2+ MB


In [20]:
News_vip_with_industrytag[["publishtime","title","tags_cn","indusrty_tags"]][40:40]

,publishtime,title,tags_cn,indusrty_tags
25,2018-08-31 14:27:13.987,《日股》日經結束連8揚；夏普摔1個月低、石油股重挫 ...,{},{}
26,2018-08-31 14:09:28.390,華為恐被迫缺席5G，三星搶出頭機會來了 ...,"{電信設備, 貨幣市場, 鴻海, 5G通訊設備, 5G, 自動駕駛車}","{車輛相關, 科技相關}"
27,2018-08-31 14:00:40.227,新興市場貨幣危機短期不利金價 但長期或為利多 ...,"{資產配置, 銀, 證券, 貨幣市場, 銀行, 黃金, 土耳其, 鈀, 南非, 鉑, 外匯,...","{金融相關, 鋼鐵金屬}"
28,2018-08-31 13:58:15.350,台/印度投保協定新約翻修 經長：努力推動於年底通過 ...,{貿易},{百貨通路}
29,2018-08-31 13:57:58.367,AI雙面刃！部分陸企狂砍40%人力、恐釀大規模失業潮 ...,{人力資源},{傳產其他}
30,2018-08-31 13:53:38.567,商用需求夯、日本PC出貨增幅遠超全球水準；蘋果慘摔 ...,"{桌上型電腦, 消費性電子產品, 筆記型電腦製造, 宏碁, 鴻海, 華碩, 筆記型電腦}",{科技相關}
31,2018-08-31 13:48:20.713,危機蔓延！印尼債狂失血、印尼盾近98年亞洲金融危機低 ...,"{貨幣市場, 東協(ASEAN), 總體經濟-美國公債, 印度, 印尼, 外匯}",{}
32,2018-08-31 13:43:59.120,美股指數期貨最新報價 13:31 ...,{},{}
33,2018-08-31 13:37:09.930,陸社保基金A股布局，Q2新建倉132檔、增持167檔 ...,"{金融, 人民幣, 基金績效, A股, PE}",{石化}
34,2018-08-31 13:31:41.380,兩岸三地500指數最新報價 13:15 ...,{人民幣},{}


#### How many articles have at least one industry tags

In [63]:
(News_vip_with_industrytag["indusrty_tags"]!=set()).value_counts()

True     282591
False    221670
Name: indusrty_tags, dtype: int64

#### Drop Atricles without Industry tags

In [68]:
pick=News_vip_with_industrytag["indusrty_tags"]!=set()
News_vip_with_at_least_one_industrytag=News_vip_with_industrytag[pick].reset_index()
News_vip_with_at_least_one_industrytag["indusrty_tags"]

0                                     {鋼鐵金屬}
1                   {百貨通路, 鋼鐵金屬, 車輛相關, 科技相關}
2                               {金融相關, 百貨通路}
3                                   {觀光休閒娛樂}
4                           {能源, 營建地產, 電機機械}
5                               {金融相關, 科技相關}
6                                     {營建地產}
7                                 {金融相關, 運輸}
8                                     {鋼鐵金屬}
9         {觀光休閒娛樂, 傳產其他, 生技醫療保健, 電機機械, 科技相關}
10                  {傳產其他, 金融相關, 營建地產, 科技相關}
11                              {金融相關, 科技相關}
12                              {傳產其他, 金融相關}
13                              {金融相關, 營建地產}
14                    {傳產其他, 鋼鐵金屬, 紡織, 車輛相關}
15                                    {車輛相關}
16                              {車輛相關, 科技相關}
17                              {金融相關, 鋼鐵金屬}
18                                    {百貨通路}
19                                    {傳產其他}
20                                    {科技相關}
21                                      {石化}
22        

##### How many articles have same body

In [91]:
df=News_vip_with_at_least_one_industrytag["body"].value_counts().reset_index().rename(columns={"index":"body","body":"counts"})
df.head(10)

,body,counts
0,<P>MoneyDJ新聞 2016-04-25 08:25:36 記者 黃文章 報導</P>...,2
1,<P>精實新聞 2011-07-28 12:54:47 記者 林詩茵 報導</P>,2
2,<P>精實新聞 2010-11-04 11:44:49 記者 林詩茵 報導</P><P>中鋼...,1
3,<P>精實新聞 2009-06-10 11:17:28 記者 余美慧 報導</P><P>新華...,1
4,<P>精實新聞 2009-10-26 08:57:26 記者 蔡承啟 報導</P><P>日經...,1
5,<P>精實新聞 2010-03-18 09:59:46 記者 郭妍希 報導</P><P>新華...,1
6,<P>精實新聞 2011-06-03 11:30:53 記者 余美慧 報導</P><P>大陸...,1
7,<P>精實新聞 2012-01-30 11:53:58 記者 許曉嘉 報導</P><P>通訊...,1
8,<P>精實新聞 2011-02-10 18:47:31 記者 羅毓嘉 報導</P><P>航空...,1
9,<P>精實新聞 2012-09-03 17:45:12 記者 陳祈儒 報導</P><P>旗下...,1


In [93]:
News_vip_with_at_least_one_industrytag[News_vip_with_at_least_one_industrytag["body"]==df["body"].values[1]]

,index,publishtime,title,body,relateid,guid,no_use,tags,title_token,body_token,from,all_1,tags_cn,indusrty_tags
189740,245245,2011-07-28 12:54:47.217,全台最貴墓地，忠孝東路54坪405萬/坪拍出(無內文) ...,<P>精實新聞 2011-07-28 12:54:47 記者 林詩茵 報導</P>,"AS9940,C025010,C025090,N0504000,",b45ccd75-e818-4413-8b0f-066365fecf15,b45ccd75-e818-4413-8b0f-066365fecf15,"C0.C025010,C0.C025090,TW.9940",全台 最貴 墓地 忠孝東路 坪 坪 拍出 內文,,1,1,"{營建, 信義, 營造工程}",{營建地產}
189741,245246,2011-07-28 12:54:47.197,台泥7月30日起全面調漲廣東、廣西水泥價格20~30元(無內文) ...,<P>精實新聞 2011-07-28 12:54:47 記者 林詩茵 報導</P>,"AS1101,C011010,",31662ec5-f4e6-4419-a8d3-4e6d64ba40ad,31662ec5-f4e6-4419-a8d3-4e6d64ba40ad,"C0.C011010,TW.1101,X1900151",台泥 日起 全面 調漲 廣東 廣西 水泥 價格 內文,,1,1,"{台泥, 水泥}",{水泥}


##### How many articles have same body_token

In [94]:
df=News_vip_with_at_least_one_industrytag["body_token"].value_counts().reset_index().rename(columns={"index":"body_token","body_token":"counts"})
df.head(10)

,body_token,counts
0,,156
1,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 32S amp P NASDA...,154
2,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX S amp P NASDAQ ...,153
3,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 30S amp P NASDA...,81
4,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 01S amp P NASDA...,81
5,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX S amp P 5001 NA...,75
6,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 32S amp P 5001 ...,69
7,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 02S amp P NASDA...,56
8,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 00S amp P NASDA...,55
9,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 29S amp P NASDA...,53


In [80]:
News_vip_with_at_least_one_industrytag[News_vip_with_at_least_one_industrytag["body_token"]==df["body_token"].values[1]]

,index,publishtime,title,body,relateid,guid,no_use,tags,title_token,body_token,from,all_1,tags_cn,indusrty_tags
256010,327927,2009-09-03 15:55:46.120,美股指數期貨最新報價 15:32 ...,<P>精實新聞 2009-09-03 15:55:46 記者 新聞編譯 報導</P><P>\...,"N0301010,N0406000,N1030200,",d2d1d231-2896-407d-ab33-bcf576263868,d2d1d231-2896-407d-ab33-bcf576263868,C0.C099094,美股 指數期貨 最新 報價,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 32S amp P NASDA...,1,1,{報紙業},{傳播出版}
256083,328011,2009-09-03 08:48:10.833,美股指數期貨最新報價 08:32 ...,<P>精實新聞 2009-09-03 08:48:10 記者 新聞編譯 報導</P><P>\...,"N0301010,N0406000,N1030200,",2e4f0cd1-b26b-449e-a4b2-5c09633d685c,2e4f0cd1-b26b-449e-a4b2-5c09633d685c,C0.C099094,美股 指數期貨 最新 報價,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 32S amp P NASDA...,1,1,{報紙業},{傳播出版}
257320,329416,2009-08-21 11:48:56.107,美股指數期貨最新報價 11:32 ...,<P>精實新聞 2009-08-21 11:48:56 記者 新聞編譯 報導</P><P>\...,"N0301010,N0406000,N1030200,",6c372129-9bf9-44fd-9138-a8d8d63880b0,6c372129-9bf9-44fd-9138-a8d8d63880b0,C0.C099094,美股 指數期貨 最新 報價,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 32S amp P NASDA...,1,1,{報紙業},{傳播出版}
257486,329604,2009-08-20 09:55:55.157,美股指數期貨最新報價 09:32 ...,<P>精實新聞 2009-08-20 09:55:55 記者 新聞編譯 報導</P><P>\...,"N0301010,N0406000,N1030200,",3f071d89-b4ce-4bf7-8f08-db919aa5cefa,3f071d89-b4ce-4bf7-8f08-db919aa5cefa,C0.C099094,美股 指數期貨 最新 報價,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 32S amp P NASDA...,1,1,{報紙業},{傳播出版}
257496,329616,2009-08-20 08:50:09.560,美股指數期貨最新報價 08:32 ...,<P>精實新聞 2009-08-20 08:50:09 記者 新聞編譯 報導</P><P>\...,"N0301010,N0406000,N1030200,",fdcef349-004a-49b6-9dbc-404b81bc2e43,fdcef349-004a-49b6-9dbc-404b81bc2e43,C0.C099094,美股 指數期貨 最新 報價,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 32S amp P NASDA...,1,1,{報紙業},{傳播出版}
257547,329678,2009-08-19 16:47:09.360,美股指數期貨最新報價 16:32 ...,<P>精實新聞 2009-08-19 16:47:09 記者 新聞編譯 報導</P><P>\...,"N0301010,N0406000,N1030200,",72b6cf61-bfb6-46f6-8398-32b3f1ff4c1e,72b6cf61-bfb6-46f6-8398-32b3f1ff4c1e,C0.C099094,美股 指數期貨 最新 報價,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 32S amp P NASDA...,1,1,{報紙業},{傳播出版}
257548,329680,2009-08-19 15:50:56.300,美股指數期貨最新報價 15:32 ...,<P>精實新聞 2009-08-19 15:50:56 記者 新聞編譯 報導</P><P>\...,"N0301010,N0406000,N1030200,",685ce6a6-f037-40e0-a2a9-875ab92905b3,685ce6a6-f037-40e0-a2a9-875ab92905b3,C0.C099094,美股 指數期貨 最新 報價,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 32S amp P NASDA...,1,1,{報紙業},{傳播出版}
257554,329686,2009-08-19 14:51:19.493,美股指數期貨最新報價 14:32 ...,<P>精實新聞 2009-08-19 14:51:19 記者 新聞編譯 報導</P><P>\...,"N0301010,N0406000,N1030200,",47c80814-0ab7-4d20-849f-313e7cc16cca,47c80814-0ab7-4d20-849f-313e7cc16cca,C0.C099094,美股 指數期貨 最新 報價,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 32S amp P NASDA...,1,1,{報紙業},{傳播出版}
257560,329693,2009-08-19 13:51:27.100,美股指數期貨最新報價 13:32 ...,<P>精實新聞 2009-08-19 13:51:27 記者 新聞編譯 報導</P><P>\...,"N0301010,N0406000,N1030200,",0217d535-12c4-4dcc-a996-81be75e0da0e,0217d535-12c4-4dcc-a996-81be75e0da0e,C0.C099094,美股 指數期貨 最新 報價,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 32S amp P NASDA...,1,1,{報紙業},{傳播出版}
257845,330007,2009-08-17 15:52:53.057,美股指數期貨最新報價 15:32 ...,<P>精實新聞 2009-08-17 15:52:53 記者 新聞編譯 報導</P><TAB...,"N0301010,N0406000,N1030200,",b33d8818-11eb-4608-ae6e-c4c556e352af,b33d8818-11eb-4608-ae6e-c4c556e352af,C0.C099094,美股 指數期貨 最新 報價,指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX 32S amp P NASDA...,1,1,{報紙業},{傳播出版}


### 新聞數牽扯到所有產業

In [123]:
dict_={k:0 for k in list_industry }
for i in tqdm_notebook(News_vip_with_at_least_one_industrytag["indusrty_tags"]):
#     print(i)
    for k in dict_ :
        if k in i : dict_[k]=dict_[k]+1
ser=pd.Series(dict_)
ser=ser.sort_values(ascending=False)
ser

科技相關      142911
鋼鐵金屬       47620
金融相關       38614
農林漁牧       20546
電機機械       19482
車輛相關       19020
傳產其他       18822
百貨通路       17752
能源         17537
營建地產       16843
公用事業       13722
生技醫療保健     13581
食品飲料       12600
觀光休閒娛樂     12586
電器電纜       12297
傳播出版       11572
運輸         10530
化學工業        9872
石化          9549
紡織          6322
建材居家用品      4995
航天軍工        3560
水泥          2295
造紙          1618
綜合           927
金屬礦採選        467
控股           202
其他             0
dtype: int64

In [124]:
##總和
ser.sum()

485842

### 新聞數牽扯到單一產業

In [120]:
#每則新聞分類
dict_=dict()
for t in News_vip_with_at_least_one_industrytag["indusrty_tags"]:
    t=",".join(list(t))
    if t not in dict_:dict_[t]=1
    else: dict_[t]+=1
#只找出單一產業新聞
dict__={k:0 for k in list_industry }
for i in pd.Series(dict_).index:
    if len(i.split(","))==1:dict__[i]=pd.Series(dict_).loc[i]

ser=pd.Series(dict__)
ser.sort_values(ascending=False)

科技相關      84507
金融相關      15791
鋼鐵金屬      13147
車輛相關       4551
生技醫療保健     4098
百貨通路       4064
營建地產       4054
傳播出版       3800
能源         2965
電機機械       2542
傳產其他       2294
觀光休閒娛樂     2105
紡織         1700
運輸         1523
化學工業       1190
公用事業       1060
食品飲料        998
石化          994
農林漁牧        746
水泥          735
造紙          717
建材居家用品      590
電器電纜        350
航天軍工        219
綜合          136
控股            8
金屬礦採選         0
其他            0
dtype: int64

In [121]:
##總和
ser.sum()

154884

# Build DataSet for Single industry News

In [158]:
# Pick up the news only involves one news and retain ["guid","title_token","body_token","indusrty_tags"] columns
_ = News_vip_with_at_least_one_industrytag["indusrty_tags"]
pick=_.progress_apply(lambda x : True if len(x)==1 else False)

DataSet=News_vip_with_at_least_one_industrytag[pick][["guid","title_token","body_token","indusrty_tags"]]
DataSet.reset_index(drop=True,inplace=True) #Reset index form 0 to ~
DataSet

,guid,title_token,body_token,indusrty_tags
0,4dd5651a-8d9d-4c38-b658-696242109941,Fund 消息 資金 連 周 回流 收債 後市 看 樂觀,美國 景氣 維持 暢旺 加上 市場 貿易戰 消息 反應 漸有 鈍化 跡象 帶動 高收益債 買...,{鋼鐵金屬}
1,8b6822c0-e646-4592-a783-0aaf54e9aa6f,安心 今年 高度 挑戰 一年 自助 點餐 機助 效率 提升,餐飲業 人事 成本 持續 攀升 基本工資 時薪將 調漲 代理 摩斯 漢堡 安心 1259 表...,{觀光休閒娛樂}
2,2a05600a-75ff-4f0e-be07-97c4362e88d6,信義 房市 北溫 南冷 大雨 民俗 澆熄 熱度,信義 房屋 統計 全台 都會區 門市 交易 表現 指出 房市 今年 農曆年 房市 相 交易 ...,{營建地產}
3,494d18ba-75d1-4276-8fb3-cccec13eebaa,陸 鋼鐵 PMI 降至 後市 鋼價料 高檔 支撐,西 本 新幹線 報導 物聯 鋼鐵 物流 專業 委員會 發布 鋼鐵行業 PMI 相較 下降 百...,{鋼鐵金屬}
4,d219315e-5b1d-41ba-b89a-7381f39c133a,巴菲特 蘋果 別 投資 特斯拉 馬斯克 太常 推文 利生 產,特斯拉 Tesla Inc. 執行長 馬斯克 Elon Musk 之前 曾 考慮 讓 公司 ...,{車輛相關}
5,4a158814-f455-4c24-9b3f-f32067dbf380,台 印度 投保 協定 新約 翻修 經長 努力 推動 於 年底 通過,有關 台灣 印度 投資 保障 協定 新約 進度 今底 是否 有機 傳出 好消息 經濟部長 沈...,{百貨通路}
6,189f4439-0d74-4881-b46c-c439804aa2e4,AI 雙面 刃 部分 陸企狂 砍 40% 人力 恐釀 大規模 失業 潮,中國 鼓勵 人工智慧 AI 自動化 不少 公司 出現 裁員 潮 40% 員工 丟掉 工作 中...,{傳產其他}
7,505cdd93-3b9e-41f2-b374-bcf5de9874b0,商用 需求 夯 日本 PC 出貨 增幅 遠超 全球 水準 蘋果 慘摔,根據 科技 研調 機構 IDC Japan 公布 統計數據 顯示 商用 需求 超夯 提振上 ...,{科技相關}
8,c6c648ca-42e2-4de3-9b35-3639e393f11b,陸 社保 基金 A股 布局 Q2 新建 倉 檔 增持 檔,上證 報 報導 大陸 隨著 上市公司 半年報 陸續 公布 被 稱 聰明 資金 社保 基金 最...,{石化}
9,fe5a239a-5978-4376-b891-913c73da3c19,台股 下跌 台幣 早盤 微 貶 分,美中 貿易 戰火 升溫 早盤 台股 尾隨 美股 下跌 不過 因逢 月底 出口商 進場 拋匯 ...,{百貨通路}


In [177]:
# Merge Title and body token
DataSet["title_token_plus_body_token"]=DataSet["title_token"]+" "+DataSet["body_token"]
DataSet["title_token_plus_body_token"][12500]

'穩懋 今年 樂觀 EPS 成長 惟 折舊 增壓 抑 毛利 空間 穩懋 半導體 上周 舉行 線上 法人 說明會 受惠 手機 產業 步入 旺季 穩懋 高層 主管 表示 公司 營運 樂觀 預期 季 營收 季 成長 17% 穩懋 首季 折舊 金額 減少 季 開始 反映 下半年 資本支出 穩懋 透露 整體 折舊 金額 年增 成 水準 預料 減少 原本 估算 毛利率 成長 空間 投資法 今年 月份 時評 穩懋 今年 營收 年增 成 站上 億元 全年 毛利率 應有 以上 水準 以新 流通 在外 股本 億元 計算 EPS 法人 看好 穩懋 今年 營收 成長 原因 主要 公司 旺季 手機 應用 PA 供貨 穩健 同時 電信 基礎建設 infrastructure 有貢獻 加上 3D感測 新產品 到位 而 晶片 高通 Qualcomm 今年 跨入 砷化鎵 PA 市場 後續 穩懋 業績 挹注 亦可 持續 關注 。（ 一 今年 折舊費 用 金額 年增 成 法人 估略降 毛利率 空間 百分點 穩懋 表示 今年 首季 部份 設備 折舊 到期 所以 當季 折舊 金額 略微 下滑 億元 下半年 新增 資本支出 預計 自第 季 開始 增加 折舊費 用會 增加 預計 折舊費 用 跟 折舊 相比 增加 成 或 成 以內 法人 預期 全年 折舊 億元 若 增加 成約 億元 折舊 將讓 原本 預估 毛利率 水準 降低 百分點 另外 穩懋 今年初 法人 說明會 提及 新產品 3D感測 接單 將於 季 出貨 不過 穩懋 3D感測 客戶 下游 終端 手機 廠拉貨量 成 產品 良率 還有 待 提升 穩懋 今年 業績 貢獻度 不大 所以 新產品 全年 穩懋 毛利率 影響度 不高 。（ 二 穩懋 Q2 營收 季增 17% 穩懋 資深 副總 陳 舜平 表示 傳統 旺季 自第 季 開始 季 預計 下半年 智慧手機 需求 本季 開始 備貨 儘管 中國 手機 市場 季稍 弱 一點 但是 並不 擔心 影響 季 表現 尤其 穩懋 客戶 群較 分散 季 成長 蠻 有信心 穩懋 表示 季 營收 預估 季增 13% 17% 外界 關心 產能 部分 陳 舜 平說 目前 總產 吋 晶圓 月產 萬片 包含 A B 合計 月產 萬片 Fab 廠約 千片 產能 跟 年底 相近 目前 產 能見度 周 即 大約 一個月 。（ 穩懋 今年 資本支出 

In [178]:
#clean token
def CleanToken(string):
    pattern=[re.compile("[\)》\，\.\、\-\%\《\(\"\'\％\」\「\。\（\）\；\● ]"),
         re.compile("\d\d%"),##EX: 85%
         re.compile("\d+"),##EX:9,10,123..
         re.compile(" [a-zA-Z] "),##EX: x , d ,...
#          re.compile("[a-zA-Z]"),
        ]
# string=trainset_vip["Title_and_body"][7000]
    for p in pattern:
        string=p.sub(" ",string)
    string=re.sub("  +"," ",string)# two or above space to one space
    string=re.sub("^ ","",string)#space at beginning
    string=re.sub(" $","",string)#space at end
    return string
DataSet["title_token_plus_body_token"]=DataSet["title_token_plus_body_token"].progress_apply(CleanToken)

In [181]:
# pick some columns ["guid","title_token_plus_body_token","indusrty_tags"]
DataSet=DataSet[["guid","title_token_plus_body_token","indusrty_tags"]]

,guid,title_token_plus_body_token,indusrty_tags
0,4dd5651a-8d9d-4c38-b658-696242109941,Fund 消息 資金 連 周 回流 收債 後市 看 樂觀 美國 景氣 維持 暢旺 加上 市場...,{鋼鐵金屬}
1,8b6822c0-e646-4592-a783-0aaf54e9aa6f,安心 今年 高度 挑戰 一年 自助 點餐 機助 效率 提升 餐飲業 人事 成本 持續 攀升 ...,{觀光休閒娛樂}
2,2a05600a-75ff-4f0e-be07-97c4362e88d6,信義 房市 北溫 南冷 大雨 民俗 澆熄 熱度 信義 房屋 統計 全台 都會區 門市 交易 ...,{營建地產}
3,494d18ba-75d1-4276-8fb3-cccec13eebaa,陸 鋼鐵 PMI 降至 後市 鋼價料 高檔 支撐 西 本 新幹線 報導 物聯 鋼鐵 物流 專...,{鋼鐵金屬}
4,d219315e-5b1d-41ba-b89a-7381f39c133a,巴菲特 蘋果 別 投資 特斯拉 馬斯克 太常 推文 利生 產 特斯拉 Tesla Inc 執...,{車輛相關}
5,4a158814-f455-4c24-9b3f-f32067dbf380,台 印度 投保 協定 新約 翻修 經長 努力 推動 於 年底 通過 有關 台灣 印度 投資 ...,{百貨通路}
6,189f4439-0d74-4881-b46c-c439804aa2e4,AI 雙面 刃 部分 陸企狂 砍 人力 恐釀 大規模 失業 潮 中國 鼓勵 人工智慧 AI ...,{傳產其他}
7,505cdd93-3b9e-41f2-b374-bcf5de9874b0,商用 需求 夯 日本 PC 出貨 增幅 遠超 全球 水準 蘋果 慘摔 根據 科技 研調 機構...,{科技相關}
8,c6c648ca-42e2-4de3-9b35-3639e393f11b,陸 社保 基金 A股 布局 新建 倉 檔 增持 檔 上證 報 報導 大陸 隨著 上市公司 半...,{石化}
9,fe5a239a-5978-4376-b891-913c73da3c19,台股 下跌 台幣 早盤 微 貶 分 美中 貿易 戰火 升溫 早盤 台股 尾隨 美股 下跌 不...,{百貨通路}


## Save DataSet

In [182]:
# # DataSet.reset_index(drop=True,inplace=True)
# with open("D:3.AutoencoderForArticle/DataSet_vip_single_industry","wb") as f:
#     pickle.dump(file=f,obj=DataSet)